__Vi bygger om logiken för att plocka ut 14 hästar när vi plockat bort de hästar som är i VNUM (1,2,3).__

In [21]:
# Common imports
import numpy as np
import os
import pandas as pd


In [22]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [23]:
# DataPiplene är en pythonfil som innehåller den egenkostruerade klassen DataFrameSelector
from DataPipeline import DataFrameSelector
from sklearn.externals import joblib

__Läser in filen som ska köras__

In [25]:
df1 = pd.read_excel('RealDataV7520190427.xlsx')

In [26]:
# Plockar bort hästara i VNUM (1,2,3) som inte ska vara med
df1 = df1[~df1.VNUM.isin([1,2,3])]    

In [27]:
df1 = df1.sort_values(['Lopp','Hast'], ascending = ['True','False']).reset_index()

In [28]:
# Importera model som ska användas
rf = joblib.load('Travmodel_v6.pkl')


In [29]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_v6.pkl')

__Definierar de variabler som ska användas i datasetet__

In [30]:

# Numerisk variabel 

num_attribs = ['VLP',
 'SVLP',
 'VSVLP',
 'VPN_SUM',
 'VPN_SUM_ORD',
 'VPK_SUM',
 'VPK_SUM_ORD',
 'VLPB',
 'SVLPB',
 'VSVLPB',
 'E_P',
 'E_P_Num',
 'E_N',
 'E_R',
 'E_U',
 'G_R',
 'A_R',
 'T_R',
 'ToR',
 'Ts_R']

__Kör modellen och plockar fram de 14 hästarna med högst score__

In [31]:
df1['GRUPP'] = df1.GRUPP.astype('object')
features = full_pipeline.fit_transform(df1)

predict = rf.predict_proba(features)

predict_cutoff = pd.DataFrame({'Prob0':predict[:,0],'Prob1':predict[:,1]}).sort_values('Prob1', ascending = False).iloc[14]['Prob1']

y_pred = np.where( predict[:,1] > predict_cutoff,1,0)

In [32]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [33]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [34]:
# Gör om till lista och stoppar in i ett dictionary
pred = y_pred.tolist()
prediction = {'Predict':pred}
# Konverterar till Dataframe
df_pred = pd.DataFrame.from_dict(prediction)

In [35]:
df_predict1 = df1.merge(df_pred, right_index = True, left_index = True)
df_predict2 = df_predict1.merge(df_scored,  right_index = True, left_index = True) 

In [36]:
df1_out = df_predict2[df_predict2.Predict == 1][['Lopp','Hast','Score']]. \
sort_values(['Lopp','Score'], ascending = [True,False])
df1_out

,Lopp,Hast,Score
7,1,11,0.096833
10,1,15,0.095272
0,1,1,0.080790
9,1,14,0.046080
12,2,2,0.047242
17,2,11,0.038039
20,3,2,0.047356
29,4,3,0.072585
27,4,1,0.052436
32,4,7,0.046437


In [37]:
len(df1_out)

14

__Export till excel__

In [39]:
df1_out.to_excel('Outtake20190427.xlsx', index = False)